In [ ]:
# coding: utf-8
import numpy as np
import os
import tensorflow as tf
if tf.__version__ < '1.6.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
import cv2
from utils import label_map_util
from utils import visualize as vis_util
import time


/Users/huanghao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
PATH_TO_CKPT = '/Users/huanghao/PycharmProjects/Jurvis/Program' \
               '/Detection/model/' \
               'my_frozen_inference_graph.pb'
PATH_TO_LABELS = '/Users/huanghao/PycharmProjects/Jurvis/Program' \
                 '/Detection/model/label_map.pbtxt'
NUM_CLASSES = 1
T0=time.time()
# ## 载入标签图
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES,
                                                            use_display_name=True)
category_index = label_map_util.create_category_index(categories)
T1=time.time()
print(T1-T0)
# ## 载入模型
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
T2=time.time()
print(T2-T1)

0.004197835922241211


1.7271521091461182


In [3]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [4]:
def inference(image):
    global sess, tensor_dict
    # The following processing is only for single image
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                           feed_dict={image_tensor: np.expand_dims(image,0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    return output_dict

In [5]:
global sess, tensor_dict
frame = cv2.imread('/Users/huanghao/PycharmProjects/Jurvis/Program/Detection/datasets/1xxx.png')
frame=cv2.resize(frame,(0,0),fx=0.5,fy=0.5)
vis_util.constant(frame,category_index)
with detection_graph.as_default():
    with tf.Session() as sess:
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
            'num_detections', 'detection_boxes', 'detection_scores',
            'detection_classes', 'detection_masks'
        ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                    tensor_name)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        # Actual detection.
        output_dict = inference(frame)
        # Visualization of the results of a detection.
        vis_util.attach_box_text_2image(
            frame,
            output_dict['num_detections'],
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
        max_boxes_to_draw=3)
        print(output_dict)
        cv2.imshow('',frame)
        flag=cv2.waitKey(0)  
        if flag==27:
            cv2.destroyAllWindows()
        pass

{'num_detections': 30, 'detection_boxes': array([[0.06810322, 0.06459397, 0.9378766 , 0.930132  ],
       [0.36884585, 0.        , 1.        , 0.6609993 ],
       [0.        , 0.        , 0.5324628 , 0.7708375 ],
       [0.        , 0.        , 0.7707459 , 0.53798807],
       [0.        , 0.25046092, 0.53921235, 1.        ],
       [0.        , 0.45117036, 0.7653972 , 1.        ],
       [0.45882607, 0.24281973, 1.        , 1.        ],
       [0.1556179 , 0.01176775, 1.        , 0.502798  ],
       [0.24476507, 0.45867077, 1.        , 1.        ],
       [0.        , 0.22134778, 1.        , 0.7735058 ],
       [0.23692423, 0.        , 0.77012235, 1.        ],
       [0.33975735, 0.        , 1.        , 0.36327255],
       [0.64363647, 0.3600349 , 1.        , 1.        ],
       [0.35590753, 0.6486938 , 1.        , 1.        ],
       [0.        , 0.        , 0.6389767 , 0.36689177],
       [0.30188066, 0.3475437 , 0.6951807 , 1.        ],
       [0.5882938 , 0.12118044, 1.        , 0.